In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageChops, ImageEnhance
import tensorflow as tf

In [9]:
# Load the trained model
model = tf.keras.models.load_model('./softmax_adam_new.keras')

In [20]:
def difference(path):
    filename = path
    # print(path)
    resaved_name = filename.split('.')[-2]+'_resaved.jpg'
    # print(resaved_name)
    resaved_name = resaved_name.split('/')[-1]
    org = Image.open(filename).convert('RGB')
    org.save(resaved_name, 'JPEG', quality=92)
    resaved = Image.open(resaved_name)
    diff = ImageChops.difference(org, resaved)
    extrema = diff.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff

    diff = ImageEnhance.Brightness(diff).enhance(scale)
    # diff
    return diff

In [21]:
# Input image path from user
path = input("Enter image path:\n")
try:
    # Open and preprocess the image
    img = Image.open(path)
    img_diff = np.array(difference(path).resize((128, 128))).flatten() / 255.0
    img_diff = img_diff.reshape(-1, 128, 128, 3)
except FileNotFoundError:
    print("Image does not exist or wrong path provided.")
    exit()

In [22]:
pred = model.predict(img_diff)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [23]:
if pred[0][0] > 0.5:
    print("Prediction: Not Forged")
else:
    print("Prediction: Forged")

Prediction: Forged
